<a href="https://colab.research.google.com/github/WaruCodes/Deep_Learning-Defect-prediction-framework-using-CodeBERT-and-GAT/blob/main/Final_Model_Black_Repository.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pandas as pd

train_df = pd.read_csv("/content/drive/MyDrive/black_only/train_black.csv")
val_df   = pd.read_csv("/content/drive/MyDrive/black_only/val_black.csv")
test_df  = pd.read_csv("/content/drive/MyDrive/black_only/test_black.csv")


In [ ]:
train_data = train_df.to_dict(orient="records")
val_data   = val_df.to_dict(orient="records")
test_data  = test_df.to_dict(orient="records")


In [ ]:
# Colab/Notebook setup
!pip -q install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu118
!pip -q install transformers==4.41.0 torchmetrics==1.4.0 scikit-learn==1.6.0 tree_sitter==0.20.2 tree_sitter_languages transformers torch
!pip -q install torch_geometric==2.6.0 torch-scatter torch-sparse -f https://data.pyg.org/whl/torch-2.3.0+cu118.html
!pip -q install tree_sitter==0.20.2

import os, sys, json, math, random, numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from transformers import RobertaTokenizer, RobertaModel
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, average_precision_score
from torchmetrics.functional import auroc

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Device:", device)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.8/137.8 kB 9.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 110.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 67.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 122.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 120.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 92.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 50.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.2 MB/s eta 0

# Data Loading and Preprocessing
Goal: Read your dataset rows, decode  as UTF‑8 (fixed file after commit), preserve exact line boundaries (including whitespace), and build labels from .

Line numbering: This notebook assumes  uses 0-based indices. If yours are 1-based, subtract 1 during preprocessing.

Commit/diff metadata: Optional functions are included to enrich samples later

In [ ]:
# ================================
# 1. Imports
# ================================
import json
import random
import pandas as pd


# ================================
# 2. YOUR ORIGINAL ADAPTER (UNCHANGED)
# ================================
def df_row_to_record(row):
    raw = row['content']

    # Step 1: Decode bytes → string
    if isinstance(raw, (bytes, bytearray)):
        content = raw.decode('utf-8', errors='ignore')
    else:
        content = str(raw)

    # Step 2: FIX escaped newlines (CRITICAL)
    if "\\n" in content:
        content = content.encode('utf-8').decode('unicode_escape')

    induce_bug = row['induce_bug']
    if isinstance(induce_bug, str):
        induce_bug = json.loads(induce_bug)

    record = {
        "datetime": row.get('datetime', ''),
        "committer": row.get('committer', ''),
        "commit": row.get('commit', ''),
        "repo": row.get('repo', ''),
        "filepath": row.get('filepath', ''),
        "content": content,           # NOW real multi-line code
        "methods": row.get('methods', []),
        "induce_bug": induce_bug,
        "file_label": int(row.get('file_label', 1 if len(induce_bug) > 0 else 0)),
    }
    return record


# ================================
# 3. LOAD REAL DATASET
# ================================
# ⟳ CHANGE THIS PATH TO YOUR DATASET
DATASET_PATH = "/content/drive/MyDrive/black_only/train_black.csv"

df = pd.read_csv(DATASET_PATH)
print("Loaded dataset size:", len(df))


# ================================
# 4. PREPROCESSING (SAFE & MINIMAL)
# ================================
def preprocess_df(df):
    # Drop rows without code content
    df = df.dropna(subset=["content"]).reset_index(drop=True)

    # Normalize induce_bug column
    def parse_induce_bug(x):
        if isinstance(x, list):
            return x
        if isinstance(x, str):
            try:
                return json.loads(x)
            except:
                return []
        return []

    df["induce_bug"] = df["induce_bug"].apply(parse_induce_bug)

    # File-level label
    if "file_label" not in df.columns:
        df["file_label"] = df["induce_bug"].apply(
            lambda x: 1 if len(x) > 0 else 0
        )

    return df


df = preprocess_df(df)
print("After preprocessing:", len(df))


# ================================
# 5. SELECT TOP-10 FILES WITH MOST BUGGY LINES
# ================================
df["bug_line_count"] = df["induce_bug"].apply(len)

top10_df = (
    df[df["bug_line_count"] > 0]
    .sort_values("bug_line_count", ascending=False)
    .head(10)
)

print("\nTop-10 buggy files:")
print(top10_df[["filepath", "bug_line_count"]])


# ================================
# 6. CONVERT TO MODEL-READY RECORDS
# ================================
records = [
    df_row_to_record(row)
    for _, row in top10_df.iterrows()
]

print("\nConverted records:", len(records))


# ================================
# 7. OPTIONAL: BATCH ITERATOR (FOR TRAINING)
# ================================
def batch_iterator(df, batch_size=16):
    for start in range(0, len(df), batch_size):
        batch = df.iloc[start:start + batch_size]
        yield [
            df_row_to_record(row)
            for _, row in batch.iterrows()
        ]


# ================================
# 8. QUICK SANITY CHECK
# ================================
sample = records[0]

print("\n--- SAMPLE RECORD ---")
print("File:", sample["filepath"])
print("Buggy lines:", sample["induce_bug"])
print("Total lines:", len(sample["content"].splitlines()))
print("File label:", sample["file_label"])


Loaded dataset size: 500
After preprocessing: 499

Top-10 buggy files:
                   filepath  bug_line_count
1      src\black\strings.py               1
4                  black.py               1
6   src\black_primer\cli.py               1
20                 setup.py               1
25    src\black\brackets.py               1
36  src\black_primer\cli.py               1
42        src\black\mode.py               1
70                 setup.py               1
74                 setup.py               1
76             docs\conf.py               1

Converted records: 10

--- SAMPLE RECORD ---
File: src\black\strings.py
Buggy lines: [90]
Total lines: 217
File label: 1


In [ ]:
len(sample["content"].splitlines())  # ✅ correct

217

In [ ]:
df_raw = pd.read_csv(DATASET_PATH)
print(df_raw["content"].isna().sum())


1


In [ ]:
lines = sample["content"].splitlines()
induce_bug = sample["induce_bug"]
len(lines) > max(induce_bug) if induce_bug else True

True

In [ ]:
print(df.columns.tolist())


['datetime', 'commit', 'repo', 'filepath', 'content', 'methods', 'induce_bug', 'file_label', 'bug_line_count']


In [ ]:
#Turn the preprocessing in to reusable pipeline

def load_and_preprocess(csv_path):
    df = pd.read_csv(csv_path)
    print(f"Loaded {csv_path}: {len(df)} rows")

    df = preprocess_df(df)
    print(f"After preprocessing: {len(df)} rows")

    return df


In [ ]:
#All splits using same pipeline

train_df = load_and_preprocess("/content/drive/MyDrive/black_only/train_black.csv")
val_df   = load_and_preprocess("/content/drive/MyDrive/black_only/val_black.csv")
test_df  = load_and_preprocess("/content/drive/MyDrive/black_only/test_black.csv")


Loaded /content/drive/MyDrive/black_only/train_black.csv: 500 rows
After preprocessing: 499 rows
Loaded /content/drive/MyDrive/black_only/val_black.csv: 29 rows
After preprocessing: 28 rows
Loaded /content/drive/MyDrive/black_only/test_black.csv: 27 rows
After preprocessing: 27 rows


In [ ]:
#Covert data frames in to records

def df_to_records(df):
    return [df_row_to_record(row) for _, row in df.iterrows()]

train_records = df_to_records(train_df)
val_records   = df_to_records(val_df)
test_records  = df_to_records(test_df)


In [ ]:
#Bug statics per splits

def print_stats(name, records):
    buggy = sum(len(r["induce_bug"]) > 0 for r in records)
    print(f"{name}: {len(records)} files | {buggy} buggy")

print_stats("Train", train_records)
print_stats("Val", val_records)
print_stats("Test", test_records)


Train: 499 files | 43 buggy
Val: 28 files | 2 buggy
Test: 27 files | 0 buggy


# Commit diff enrichment
This step shows how you could fetch commit diffs and highlight changed lines. It’s optional and may require auth for large-scale use.

In [ ]:
import re
import requests

def fetch_github_diff(owner, repo, commit_hash):
    """
    Fetch unified diff for a public GitHub repo commit.
    Returns raw diff text or None on failure.
    """
    url = f"https://github.com/{owner}/{repo}/commit/{commit_hash}.diff"
    try:
        r = requests.get(url, timeout=10)
        if r.status_code == 200:
            return r.text
    except Exception as e:
        return None
    return None

def parse_changed_lines_from_diff(diff_text, target_filepath):
    """
    Very light parser to extract changed line numbers (post-commit) for a single file.
    Parses hunk headers like @@ -a,b +c,d @@ and counts added lines in that hunk.
    Returns set of 0-based indices for lines in the post-commit file that were changed.
    """
    if diff_text is None:
        return set()
    changed = set()
    current_file = None
    for line in diff_text.splitlines():
        if line.startswith("diff --git"):
            current_file = None
        elif line.startswith("+++ b/"):
            current_file = line[len("+++ b/"):].strip()
        elif current_file == target_filepath and line.startswith("@@"):
            # Example: @@ -10,7 +10,9 @@
            m = re.search(r"\+(\d+),?(\d+)?", line)
            if not m:
                continue
            start = int(m.group(1))
            count = int(m.group(2)) if m.group(2) else 1
            # Track indices in the post-commit file; subtract 1 for 0-based
            for idx in range(start-1, start-1+count):
                changed.add(idx)
        # Note: A robust parser should also handle '+' lines inside hunks to mark added lines.
    return changed

# Example usage (public repo): owner='psf', repo='black', commit='00e7e12a3a412ea386806d5d4eeaed345e912940'
# diff = fetch_github_diff('psf', 'black', '00e7e12a3a412ea386806d5d4eeaed345e912940')
# changed = parse_changed_lines_from_diff(diff, 'src/black/linegen.py')
# print("Changed lines (post-commit, 0-based):", sorted(list(changed))[:20])

In [ ]:
# Example usage: fetch diff and print changed lines

owner = "psf"
repo = "black"
commit_hash = "00e7e12a3a412ea386806d5d4eeaed345e912940"
target_filepath = "src/black/linegen.py"

diff = fetch_github_diff(owner, repo, commit_hash)

if diff is None:
    print("Failed to fetch diff")
else:
    changed = parse_changed_lines_from_diff(diff, target_filepath)
    print("Changed lines (post-commit, 0-based):")
    print(sorted(changed))


Changed lines (post-commit, 0-based):
[225, 226, 227, 228, 229, 230, 231, 232, 233]


# Line tokenization and CodeBERT embeddings
Keep whitespace: We never strip or trim lines. We split by  exactly to preserve  alignment.
Embedding strategy: Use CodeBERT’s [CLS] embedding per line. Then contextualize with BiLSTM across lines to capture neighbors.

In [ ]:
# CodeBERT
tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base")
codebert = RobertaModel.from_pretrained("microsoft/codebert-base").to(device)
codebert.eval()

@torch.no_grad()
def embed_lines_with_codebert(lines, max_len=128):
    # Returns tensor [L, 768]; preserves line indices and whitespace alignment
    embs = []
    for line in lines:
        tokens = tokenizer.encode(line, truncation=True, max_length=max_len)
        input_ids = torch.tensor(tokens, dtype=torch.long, device=device).unsqueeze(0)
        outputs = codebert(input_ids)
        cls_emb = outputs.last_hidden_state[:, 0, :]  # [1, 768]
        embs.append(cls_emb.squeeze(0))
    return torch.stack(embs, dim=0)

# BiLSTM contextualizer
class ContextEncoder(nn.Module):
    def __init__(self, in_dim=768, hidden=256, num_layers=1, dropout=0.2):
        super().__init__()
        self.lstm = nn.LSTM(in_dim, hidden, num_layers=num_layers,
                            bidirectional=True, batch_first=True, dropout=dropout)
        self.proj = nn.Linear(hidden*2, hidden*2)
        self.norm = nn.LayerNorm(hidden*2)

    def forward(self, x):  # x: [B, L, 768]
        h, _ = self.lstm(x)
        y = self.norm(torch.tanh(self.proj(h)))  # [B, L, 512]
        return y

context_encoder = ContextEncoder().to(device)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/498 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(


# Graph construction: AST + CFG-lite + PDG-lite → CPG
AST: Tree-sitter Python grammar to gather node spans. We link consecutive lines inside the same span.

CFG-lite: Simple sequential edges line i ↔ i+1. This keeps the graph sparse and Colab-friendly.

PDG-lite: Connect lines that share identifier names (excluding Python keywords), within a local window.

In [ ]:

!pip install tree-sitter tree-sitter-languages
!pip install torch
from tree_sitter import Language, Parser
from tree_sitter_languages import get_language
import torch


# Build Python grammar

from tree_sitter import Parser

PY_LANGUAGE = get_language("python")

parser = Parser()
parser.set_language(PY_LANGUAGE)


def parse_ast_spans(code):
    tree = parser.parse(code.encode("utf8"))
    root = tree.root_node
    spans = []

    def visit(node):
        spans.append((node.type, node.start_point[0], node.end_point[0]))  # 0-based lines
        for child in node.children:
            visit(child)

    visit(root)
    return spans


def extract_identifiers_python(line):
    tokens = [t for t in ''.join([c if (c.isalnum() or c=='_') else ' ' for c in line]).split() if len(t) > 1]
    stop = set([
        "def","class","return","if","else","elif","for","while","import","from","as","with",
        "try","except","finally","True","False","None","and","or","not","in","is","lambda",
        "global","nonlocal","assert","yield","pass","break","continue","raise"
    ])
    return [t for t in tokens if t not in stop]

def build_cpg(lines, max_window=15):
    num_lines = len(lines)
    edge_index = []
    edge_type = []  # 0=sequential, 1=AST-span adjacency, 2=def-use PDG-lite

    # CFG-lite: sequential edges
    for i in range(num_lines-1):
        edge_index.append([i, i+1]); edge_type.append(0)
        edge_index.append([i+1, i]); edge_type.append(0)

    # AST-lite: adjacency inside span ranges
    code = "\n".join(lines)
    spans = parse_ast_spans(code)
    for tp, s, e in spans:
        s = max(0, min(s, num_lines-1))
        e = max(0, min(e, num_lines-1))
        for i in range(s, e):
            edge_index.append([i, i+1]); edge_type.append(1)
            edge_index.append([i+1, i]); edge_type.append(1)

    # PDG-lite: co-occurrence window edges for identifiers
    id_map = {}
    for i, line in enumerate(lines):
        ids = extract_identifiers_python(line)
        for tok in ids:
            id_map.setdefault(tok, []).append(i)
    for tok, idxs in id_map.items():
        idxs = sorted(set(idxs))
        for a in idxs:
            for b in idxs:
                if a != b and abs(a-b) <= max_window:
                    edge_index.append([a, b]); edge_type.append(2)

    edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous() if edge_index else torch.empty((2,0), dtype=torch.long)
    edge_type = torch.tensor(edge_type, dtype=torch.long) if edge_type else torch.empty((0,), dtype=torch.long)
    return edge_index, edge_type

# Dataset class
Function: Turn each record into a graph sample:

CodeBERT embeddings → BiLSTM contextualization → node features

Build CPG-lite edges

Prepare line labels from  (exact indices preserved)

File label for the file-level head

In [ ]:
from torch.utils.data import Dataset, DataLoader

# -----------------------------
# Dataset wrapper (unchanged)
# -----------------------------
class LineGraphSample:
    def __init__(self, x, edge_index, edge_type, line_labels, file_label, mask_lines):
        self.x = x
        self.edge_index = edge_index
        self.edge_type = edge_type
        self.line_labels = line_labels
        self.file_label = file_label
        self.mask_lines = mask_lines


class DefectDataset(Dataset):
    def __init__(self, records, cache=True):
        self.records = records
        self.cache = cache
        self._cache = [None] * len(records)

    def __len__(self):
        return len(self.records)

    def __getitem__(self, idx):
        if self.cache and self._cache[idx] is not None:
            return self._cache[idx]

        rec = self.records[idx]
        lines = rec["content"].split("\n")   # keep original line indices
        L = len(lines)

        with torch.no_grad():
            line_embs = embed_lines_with_codebert(lines)   # [L, 768]

        ctx_inp = line_embs.unsqueeze(0)                   # [1, L, 768]
        ctx_out = context_encoder(ctx_inp).squeeze(0)      # [L, 512]

        edge_index, edge_type = build_cpg(lines)

        # ----- line-level labels (0-based) -----
        line_labels = torch.zeros(L, dtype=torch.float32)
        for ln in rec["induce_bug"]:
            if 0 <= ln < L:
                line_labels[ln] = 1.0

        file_label = torch.tensor(rec["file_label"], dtype=torch.long)
        mask_lines = torch.ones(L, dtype=torch.bool)

        sample = LineGraphSample(
            x=ctx_out,
            edge_index=edge_index,
            edge_type=edge_type,
            line_labels=line_labels,
            file_label=file_label,
            mask_lines=mask_lines
        )

        if self.cache:
            self._cache[idx] = sample

        return sample


In [ ]:
# Split data and create DataLoaders
from torch.utils.data import DataLoader

def collate(samples):
    return samples  # one file per batch

train_ds = DefectDataset(train_records, cache=True)
val_ds   = DefectDataset(val_records,   cache=False)
test_ds  = DefectDataset(test_records,  cache=False)

train_loader = DataLoader(
    train_ds,
    batch_size=1,
    shuffle=True,
    collate_fn=collate
)

val_loader = DataLoader(
    val_ds,
    batch_size=1,
    shuffle=False,
    collate_fn=collate
)

test_loader = DataLoader(
    test_ds,
    batch_size=1,
    shuffle=False,
    collate_fn=collate
)

print("Train:", len(train_ds))
print("Val:  ", len(val_ds))
print("Test: ", len(test_ds))



Train: 499
Val:   28
Test:  27


# GAT fusion and dual-head model

Edge-type embeddings: Inject coarse edge-type info into GAT via an augmentation trick.

Residuals and normalization: Keep features stable and avoid over-smoothing.

Heads:

Line-level sigmoid for per-line probabilities
File-level attention pooling + MLP for buggy vs clean

In [ ]:
from torch_geometric.nn import GATConv, GlobalAttention

class LineDefectModel(nn.Module):
    def __init__(self, in_dim=512, hidden=256, heads1=8, heads2=4, num_edge_types=3, dropout=0.2):
        super().__init__()
        self.edge_emb = nn.Embedding(num_edge_types, 16)
        self.gat1 = GATConv(in_dim + 16, hidden, heads=heads1, dropout=dropout, concat=True)
        self.gat2 = GATConv(hidden*heads1 + 16, hidden, heads=heads2, dropout=dropout, concat=True)
        out_dim = hidden*heads2
        self.norm = nn.LayerNorm(out_dim)
        self.line_head = nn.Linear(out_dim, 1)
        self.gate_nn = nn.Sequential(nn.Linear(out_dim, 64), nn.ReLU(), nn.Linear(64, 1))
        self.global_att = GlobalAttention(gate_nn=self.gate_nn)
        self.file_head = nn.Sequential(nn.Linear(out_dim, 256), nn.ReLU(), nn.Dropout(dropout), nn.Linear(256, 2))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, edge_index, edge_type, mask_lines=None):
        if edge_index.numel() == 0:
            h = self.norm(x)
        else:
            e = self.edge_emb(edge_type)
            src, dst = edge_index
            # Aggregate edge-type embeddings to source nodes (approximate edge feature injection)
            aug1 = torch.zeros((x.size(0), e.size(1)), device=x.device)
            aug1.index_add_(0, src, e)
            x1 = torch.cat([x, aug1], dim=1)
            h1 = self.gat1(x1, edge_index)
            h1 = F.elu(h1)
            # Second layer with updated augmentation
            aug2 = torch.zeros((h1.size(0), e.size(1)), device=h1.device)
            aug2.index_add_(0, src, e)
            x2 = torch.cat([h1, aug2], dim=1)
            h2 = self.gat2(x2, edge_index)
            h = self.norm(h2)

        h = self.dropout(h)
        line_logits = self.line_head(h).squeeze(-1)  # [L]
        if mask_lines is None:
            mask_lines = torch.ones(h.size(0), dtype=torch.bool, device=h.device)
        file_repr = self.global_att(h[mask_lines])   # [D]
        file_logits = self.file_head(file_repr)      # [2]
        return line_logits, file_logits

/usr/local/lib/python3.12/dist-packages/torch_geometric/typing.py:86: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: Could not load this library: /usr/local/lib/python3.12/dist-packages/torch_scatter/_version_cuda.so
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
/usr/local/lib/python3.12/dist-packages/torch_geometric/typing.py:124: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: Could not load this library: /usr/local/lib/python3.12/dist-packages/torch_sparse/_version_cuda.so
  warnings.warn(f"An issue occurred while importing 'torch-sparse'. "


# Losses, optimizer, and training loop

Focal loss: For line-level imbalance.

Multi-task loss: Emphasize line-level learning while keeping file-level useful.

Gradient clipping: Stability.

In [ ]:
def focal_loss(prob, target, alpha=0.25, gamma=2.0, eps=1e-6):
    prob = torch.clamp(prob, eps, 1.0 - eps)
    ce_pos = -torch.log(prob)
    ce_neg = -torch.log(1.0 - prob)
    loss = alpha * (1 - prob)**gamma * target * ce_pos + (1 - alpha) * (prob**gamma) * (1 - target) * ce_neg
    return loss.mean()

lambda_line, lambda_file = 2.0, 1.0
model = LineDefectModel().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)

def train_epoch(loader):
    model.train()
    total = 0.0
    for batch in loader:
        sample = batch[0]  # single sample per batch
        x = sample.x.to(device)
        edge_index = sample.edge_index.to(device)
        edge_type = sample.edge_type.to(device)
        line_labels = sample.line_labels.to(device)
        file_label = sample.file_label.to(device)
        mask_lines = sample.mask_lines.to(device)

        optimizer.zero_grad()
        line_logits, file_logits = model(x, edge_index, edge_type, mask_lines)
        line_prob = torch.sigmoid(line_logits)
        line_loss = focal_loss(line_prob, line_labels)

        # Convert file_label to one-hot encoding for F.cross_entropy
        num_classes = 2 # Assuming 2 classes: buggy and non-buggy
        file_label_one_hot = F.one_hot(file_label, num_classes=num_classes).float()
        file_loss = F.cross_entropy(file_logits.unsqueeze(0), file_label_one_hot.unsqueeze(0))
        file_loss = F.cross_entropy(file_logits, file_label.unsqueeze(0))


        loss = lambda_line * line_loss + lambda_file * file_loss
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        total += loss.item()
    return total / max(1, len(loader))

/usr/local/lib/python3.12/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'nn.glob.GlobalAttention' is deprecated, use 'nn.aggr.AttentionalAggregation' instead
  warnings.warn(out)


# Evaluation metrics

File-level: Accuracy, F1, ROC-AUC, PR-AUC

Line-level: PR-AUC (global), plus Top‑k and MRR for localization

In [ ]:
def evaluate(loader):
    model.eval()
    file_probs, file_labels = [], []
    line_probs, line_labels = [], []

    with torch.no_grad():
        for batch in loader:
            sample = batch[0]
            x = sample.x.to(device)
            edge_index = sample.edge_index.to(device)
            edge_type = sample.edge_type.to(device)
            mask_lines = sample.mask_lines.to(device)

            line_logits, file_logits = model(x, edge_index, edge_type, mask_lines)
            fp = torch.softmax(file_logits, dim=-1)[1].item()
            file_probs.append(fp)
            file_labels.append(sample.file_label.item())

            lp = torch.sigmoid(line_logits).cpu().numpy().tolist()
            ll = sample.line_labels.cpu().numpy().tolist()
            line_probs.extend(lp)
            line_labels.extend(ll)

    # File-level
    file_probs_np = np.array(file_probs)
    file_labels_np = np.array(file_labels)
    file_preds_bin = (file_probs_np >= 0.5).astype(int)

    acc = accuracy_score(file_labels_np, file_preds_bin)
    f1 = f1_score(file_labels_np, file_preds_bin)
    try:
        roc = roc_auc_score(file_labels_np, file_probs_np)
    except Exception:
        roc = float('nan')
    pr = average_precision_score(file_labels_np, file_probs_np)

    # Line-level (global PR-AUC across all lines)
    line_probs_np = np.array(line_probs)
    line_labels_np = np.array(line_labels)
    l_pr = average_precision_score(line_labels_np, line_probs_np)

    return {
        "file": {"acc": acc, "f1": f1, "roc": roc, "pr": pr},
        "line": {"pr": l_pr}
    }

def topk_mrr(loader, k=5):
    model.eval()
    hits, rr_list = [], []
    with torch.no_grad():
        for batch in loader:
            sample = batch[0]
            x = sample.x.to(device)
            edge_index = sample.edge_index.to(device)
            edge_type = sample.edge_type.to(device)
            mask_lines = sample.mask_lines.to(device)

            line_logits, _ = model(x, edge_index, edge_type, mask_lines)
            probs = torch.sigmoid(line_logits).cpu().numpy()
            buggy_idx = np.where(sample.line_labels.cpu().numpy() > 0.5)[0]
            if len(buggy_idx) == 0:
                continue
            order = np.argsort(-probs)
            topk = order[:k]
            hit = int(any(b in topk for b in buggy_idx))
            hits.append(hit)
            ranks = [np.where(order == b)[0][0] + 1 for b in buggy_idx]
            rr = 1.0 / min(ranks)
            rr_list.append(rr)
    hit_rate = np.mean(hits) if hits else 0.0
    mrr = np.mean(rr_list) if rr_list else 0.0
    return hit_rate, mrr

#Train and Validate

For real datasets, increase epochs (e.g., 20–40), and consider caching embeddings to disk for speed.

In [ ]:
epochs = 6

for ep in range(1, epochs + 1):
    tr_loss = train_epoch(train_loader)
    val_metrics = evaluate(val_loader)

    print(
        f"Epoch {ep:02d} | "
        f"TrainLoss {tr_loss:.4f} | "
        f"Val File("
        f"acc {val_metrics['file']['acc']:.3f}, "
        f"f1 {val_metrics['file']['f1']:.3f}, "
        f"roc {val_metrics['file']['roc']:.3f}, "
        f"pr {val_metrics['file']['pr']:.3f}"
        f") | "
        f"Line (pr {val_metrics['line']['pr']:.3f})"
    )


RuntimeError: expected scalar type Long but found Float

In [ ]:
import torch
from tree_sitter import Parser
from tree_sitter_languages import get_language

parser = Parser()
PY_LANGUAGE = get_language("python")
parser.set_language(PY_LANGUAGE)

print("Tree-sitter Python loaded ✅")

Tree-sitter Python loaded ✅


In [ ]:
#AST parsing (line spans)
def parse_ast_nodes(code: str):
    tree = parser.parse(bytes(code, "utf8"))
    root = tree.root_node
    spans = []

    def visit(node):
        spans.append((node.type, node.start_point[0], node.end_point[0]))
        for child in node.children:
            visit(child)

    visit(root)
    return spans


In [ ]:
#CPG-lite builder (CFG + AST + PDG)
def build_cpg(lines):
    num_lines = len(lines)
    edge_index = []
    edge_type = []

    # CFG-lite
    for i in range(num_lines - 1):
        edge_index += [[i, i+1], [i+1, i]]
        edge_type += [0, 0]

    # AST-lite
    spans = parse_ast_nodes("\n".join(lines))
    for _, s, e in spans:
        for i in range(max(0, s), min(e, num_lines - 1)):
            edge_index += [[i, i+1], [i+1, i]]
            edge_type += [1, 1]

    # PDG-lite
    def extract_ids(line):
        toks = ''.join(c if c.isalnum() or c=='_' else ' ' for c in line).split()
        stop = {
            "def","class","return","if","else","for","while","import","from",
            "True","False","None","and","or","not"
        }
        return [t for t in toks if len(t) > 1 and t not in stop]

    id_map = {}
    for i, line in enumerate(lines):
        for t in extract_ids(line):
            id_map.setdefault(t, []).append(i)

    for idxs in id_map.values():
        idxs = list(set(idxs))
        for a in idxs:
            for b in idxs:
                if a != b and abs(a-b) <= 15:
                    edge_index.append([a,b])
                    edge_type.append(2)

    return (
        torch.tensor(edge_index).t().contiguous(),
        torch.tensor(edge_type)
    )


In [ ]:
#CodeBERT line embeddings
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base")
model = RobertaModel.from_pretrained("microsoft/codebert-base").to(device)
model.eval()

@torch.no_grad()
def encode_lines(lines):
    inputs = tokenizer(
        lines, padding=True, truncation=True,
        max_length=128, return_tensors="pt"
    ).to(device)
    out = model(**inputs)
    return out.last_hidden_state[:,0,:].cpu()


In [ ]:
import pandas as pd
import ast

train_df = pd.read_csv("/content/drive/MyDrive/black_only/train_black.csv")

# Convert DataFrame → list of dicts
train_data = train_df.to_dict(orient="records")

# Fix 'Induce bug' column and add 'file_label'
for r in train_data:
    # Handle 'Induce bug' column: if NaN or malformed string, default to empty list
    if isinstance(r["Induce bug"], str):
        cleaned_str_content = r["Induce bug"].strip('[] ')
        if cleaned_str_content:
            parts = cleaned_str_content.split()
            formatted_list_str = f"[{', '.join(parts)}]"
        else:
            formatted_list_str = "[]"
        try:
            r["Induce bug"] = ast.literal_eval(formatted_list_str)
        except (ValueError, SyntaxError):
            r["Induce bug"] = [] # Fallback to empty list if parsing fails
    elif pd.isna(r["Induce bug"]):
        r["Induce bug"] = []

    # Add 'file_label' if missing, defaulting to 0
    if "file_label" not in r:
        r["file_label"] = 0


In [ ]:
# ===== End-to-end test using dataset record (FIXED) =====

rec = train_data[0]

# Fix escaped newlines
code = rec["content"].encode("utf-8").decode("unicode_escape")
lines = code.split("\n")

with torch.no_grad():
    embs = encode_lines(lines)

edges, etypes = build_cpg(lines)
etypes = etypes.long()

print("Repository:", rec["repo"])
print("File path:", rec["filepath"])
print("Commit:", rec["commit"])
print("Total lines:", len(lines))
print("Buggy lines:", rec["Induce bug"])

print("\nLine embeddings shape:", embs.shape)
print("Edges shape:", edges.shape)
print("Edge types distribution:", torch.bincount(etypes))

print("\n✅ FULL PIPELINE WORKING ON REAL DATA")

Repository: black
File path: src\black\linegen.py
Commit: 00e7e12a3a412ea386806d5d4eeaed345e912940
Total lines: 791
Buggy lines: []

Line embeddings shape: torch.Size([791, 768])
Edges shape: torch.Size([2, 17474])
Edge types distribution: tensor([1580, 9992, 5902])

✅ FULL PIPELINE WORKING ON REAL DATA


In [ ]:
#Dataset class: building samples end-to-end
#We embed lines, contextualize, build graphs, and prepare labels per file

class LineGraphSample:
    def __init__(self, x, edge_index, edge_type, line_labels, file_label, mask_lines):
        self.x = x                # [L, Feat]
        self.edge_index = edge_index  # [2, E]
        self.edge_type = edge_type    # [E]
        self.line_labels = line_labels  # [L]
        self.file_label = file_label    # scalar
        self.mask_lines = mask_lines    # boolean mask for pooling

class DefectDataset(Dataset):
    def __init__(self, records, cache=False):
        self.records = records
        self.cache = cache
        self._cache = [None]*len(records)

    def __len__(self): return len(self.records)

    def __getitem__(self, idx):
        if self.cache and self._cache[idx] is not None:
            return self._cache[idx]
        rec = self.records[idx]
        lines = rec["content"].split("\n")
        # Step 1: CodeBERT line embeddings
        with torch.no_grad():
            line_embs = embed_lines_with_codebert(lines)  # [L, 768]
        # Step 2: Contextualization
        ctx_inp = line_embs.unsqueeze(0)  # [1, L, 768]
        ctx_out = context_encoder(ctx_inp).squeeze(0)  # [L, 512]
        # Step 3: Graph construction (CPG-lite)
        edge_index, edge_type = build_cpg(lines)
        # Step 4: labels
        L = len(lines)
        line_labels = torch.zeros(L, dtype=torch.float32)
        for ln in rec["induce_bug"]:
            if 0 <= ln < L:
                line_labels[ln] = 1.0
        file_label = torch.tensor(rec["file_label"], dtype=torch.long)
        mask_lines = torch.ones(L, dtype=torch.bool)  # all lines included
        sample = LineGraphSample(ctx_out, edge_index, edge_type, line_labels, file_label, mask_lines)
        if self.cache:
            self._cache[idx] = sample
        return sample

# Split data
random.shuffle(data)
train_split = int(0.7 * len(data))
val_split = int(0.85 * len(data))
train_ds = DefectDataset(data[:train_split], cache=True)
val_ds = DefectDataset(data[train_split:val_split], cache=False)
test_ds = DefectDataset(data[val_split:], cache=False)

def collate(samples):
    # We will process each file separately (no graph batching to keep things simple in Colab)
    return samples

train_loader = DataLoader(train_ds, batch_size=1, shuffle=True, collate_fn=collate)
val_loader = DataLoader(val_ds, batch_size=1, shuffle=False, collate_fn=collate)
test_loader = DataLoader(test_ds, batch_size=1, shuffle=False, collate_fn=collate)
len(train_ds), len(val_ds), len(test_ds)

NameError: name 'data' is not defined

In [ ]:
#Model: GAT fusion + dual heads (line and file)
#We use PyTorch Geometric’s GATConv. Edge types are handled via learned type embeddings concatenated to messages.

from torch_geometric.nn import GATConv, GlobalAttention

class LineDefectModel(nn.Module):
    def __init__(self, in_dim=512, hidden=256, heads1=8, heads2=4, num_edge_types=3, dropout=0.2):
        super().__init__()
        self.edge_emb = nn.Embedding(num_edge_types, 16)
        self.gat1 = GATConv(in_dim + 16, hidden, heads=heads1, dropout=dropout, concat=True)
        self.gat2 = GATConv(hidden*heads1 + 16, hidden, heads=heads2, dropout=dropout, concat=True)
        out_dim = hidden*heads2
        self.norm = nn.LayerNorm(out_dim)
        self.line_head = nn.Linear(out_dim, 1)
        self.gate_nn = nn.Sequential(nn.Linear(out_dim, 64), nn.ReLU(), nn.Linear(64, 1))
        self.global_att = GlobalAttention(gate_nn=self.gate_nn)
        self.file_head = nn.Sequential(nn.Linear(out_dim, 256), nn.ReLU(), nn.Dropout(dropout), nn.Linear(256, 2))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, edge_index, edge_type, mask_lines=None):
        if edge_index.numel() == 0:
            # No edges: fall back to MLP
            h = self.norm(x)
        else:
            e = self.edge_emb(edge_type)
            # Expand edge-type features to source nodes (message trick)
            # GATConv doesn't natively take edge features; we concatenate edge-type emb to node features per edge via indexing
            # We implement by augmenting node features replicated along edges:
            # For implementation simplicity in single-file batches, we attach edge-type emb to the source node features at aggregation time:
            # Create augmented node features by summing edge-type embeddings for incoming edges (approximation).
            aug = torch.zeros((x.size(0), e.size(1)), device=x.device)
            src, dst = edge_index
            aug.index_add_(0, src, e)  # aggregate edge-type emb to source
            x1 = torch.cat([x, aug], dim=1)
            h1 = self.gat1(x1, edge_index)
            h1 = F.elu(h1)
            # second layer
            aug2 = torch.zeros((h1.size(0), e.size(1)), device=h1.device)
            aug2.index_add_(0, src, e)
            x2 = torch.cat([h1, aug2], dim=1)
            h2 = self.gat2(x2, edge_index)
            h = self.norm(h2)

        h = self.dropout(h)
        line_logits = self.line_head(h).squeeze(-1)  # [L]
        if mask_lines is None:
            mask_lines = torch.ones(h.size(0), dtype=torch.bool, device=h.device)
        file_repr = self.global_att(h[mask_lines])   # [D]
        file_logits = self.file_head(file_repr)      # [2]
        return line_logits, file_logits

In [ ]:
#Losses: focal loss for line-level, CE for file-leve
def focal_loss(prob, target, alpha=0.25, gamma=2.0, eps=1e-6):
    # prob: [L], target: [L] in {0,1}
    prob = torch.clamp(prob, eps, 1.0 - eps)
    ce_pos = -torch.log(prob)
    ce_neg = -torch.log(1.0 - prob)
    loss = alpha * (1 - prob)**gamma * target * ce_pos + (1 - alpha) * (prob**gamma) * (1 - target) * ce_neg
    return loss.mean()

lambda_line, lambda_file = 2.0, 1.0
model = LineDefectModel().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)

In [ ]:
#Training loop
def train_epoch(loader):
    model.train()
    total = 0.0
    for batch in loader:
        # single-sample batches for simplicity
        sample = batch[0]
        x = sample.x.to(device)
        edge_index = sample.edge_index.to(device)
        edge_type = sample.edge_type.to(device)
        line_labels = sample.line_labels.to(device)
        file_label = sample.file_label.to(device)
        mask_lines = sample.mask_lines.to(device)

        optimizer.zero_grad()
        line_logits, file_logits = model(x, edge_index, edge_type, mask_lines)
        line_prob = torch.sigmoid(line_logits)
        line_loss = focal_loss(line_prob, line_labels)
        file_loss = F.cross_entropy(file_logits.unsqueeze(0), file_label.unsqueeze(0))
        loss = lambda_line * line_loss + lambda_file * file_loss
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        total += loss.item()
    return total / len(loader)

def evaluate(loader):
    model.eval()
    file_probs, file_labels = [], []
    line_probs, line_labels = [], []

    with torch.no_grad():
        for batch in loader:
            sample = batch[0]
            x = sample.x.to(device)
            edge_index = sample.edge_index.to(device)
            edge_type = sample.edge_type.to(device)
            mask_lines = sample.mask_lines.to(device)

            line_logits, file_logits = model(x, edge_index, edge_type, mask_lines)
            fp = torch.softmax(file_logits, dim=-1)[1].item()
            file_probs.append(fp)
            file_labels.append(sample.file_label.item())

            lp = torch.sigmoid(line_logits).detach().cpu().numpy().tolist()
            ll = sample.line_labels.detach().cpu().numpy().tolist()
            line_probs.extend(lp)
            line_labels.extend(ll)

    # File-level metrics
    file_probs_np = np.array(file_probs)
    file_labels_np = np.array(file_labels)
    file_preds_bin = (file_probs_np >= 0.5).astype(int)

    acc = accuracy_score(file_labels_np, file_preds_bin)
    f1 = f1_score(file_labels_np, file_preds_bin)
    try:
        roc = roc_auc_score(file_labels_np, file_probs_np)
    except:
        roc = float('nan')
    pr = average_precision_score(file_labels_np, file_probs_np)

    # Line-level metrics (PR-AUC)
    line_probs_np = np.array(line_probs)
    line_labels_np = np.array(line_labels)
    l_pr = average_precision_score(line_labels_np, line_probs_np)

    return {
        "file": {"acc": acc, "f1": f1, "roc": roc, "pr": pr},
        "line": {"pr": l_pr}
    }

# Train
epochs = 6
for ep in range(1, epochs+1):
    tr_loss = train_epoch(train_loader)
    val_metrics = evaluate(val_loader)
    print(f"Epoch {ep:02d} | TrainLoss {tr_loss:.4f} | "
          f"Val File(acc {val_metrics['file']['acc']:.3f}, f1 {val_metrics['file']['f1']:.3f}, "
          f"roc {val_metrics['file']['roc']:.3f}, pr {val_metrics['file']['pr']:.3f}) | "
          f"Line (pr {val_metrics['line']['pr']:.3f})")

In [ ]:
#Top-k hit and MRR for line-level localization

def topk_mrr(loader, k=5):
    model.eval()
    hits, rr_list = [], []
    with torch.no_grad():
        for batch in loader:
            sample = batch[0]
            x = sample.x.to(device)
            edge_index = sample.edge_index.to(device)
            edge_type = sample.edge_type.to(device)
            mask_lines = sample.mask_lines.to(device)

            line_logits, _ = model(x, edge_index, edge_type, mask_lines)
            probs = torch.sigmoid(line_logits).detach().cpu().numpy()
            buggy_idx = np.where(sample.line_labels.cpu().numpy() > 0.5)[0]
            if len(buggy_idx) == 0:
                continue
            # Rank lines by prob
            order = np.argsort(-probs)
            topk = order[:k]
            hit = int(any(b in topk for b in buggy_idx))
            hits.append(hit)
            # Reciprocal rank
            ranks = [np.where(order == b)[0][0] + 1 for b in buggy_idx]
            rr = 1.0 / min(ranks)
            rr_list.append(rr)
    hit_rate = np.mean(hits) if hits else 0.0
    mrr = np.mean(rr_list) if rr_list else 0.0
    return hit_rate, mrr

test_metrics = evaluate(test_loader)
hit5, mrr = topk_mrr(test_loader, k=5)

print(f"Test File: Acc={test_metrics['file']['acc']:.3f} F1={test_metrics['file']['f1']:.3f} "
      f"ROC={test_metrics['file']['roc']:.3f} PR={test_metrics['file']['pr']:.3f}")
print(f"Test Line: PR-AUC={test_metrics['line']['pr']:.3f} Top-5 Hit={hit5:.3f} MRR={mrr:.3f}")

In [ ]:
# Inference and visualization: line scores per file


def inspect_sample(sample_idx=0, loader=test_loader):
    sample = list(loader)[sample_idx][0]
    x = sample.x.to(device)
    edge_index = sample.edge_index.to(device)
    edge_type = sample.edge_type.to(device)
    mask_lines = sample.mask_lines.to(device)
    lines = data[val_split + sample_idx]["content"].split("\n")
    with torch.no_grad():
        line_logits, file_logits = model(x, edge_index, edge_type, mask_lines)
        probs = torch.sigmoid(line_logits).cpu().numpy()
        file_prob = torch.softmax(file_logits, dim=-1)[1].item()
    ranked = np.argsort(-probs)
    print(f"File buggy prob: {file_prob:.3f} | True label: {sample.file_label.item()}")
    print("Top-10 lines:")
    for r in ranked[:10]:
        flag = ("<BUG>" if sample.line_labels[r].item() > 0.5 else "")
        print(f"{r:4d}  prob={probs[r]:.3f}  {flag}  | {lines[r][:100]}")

inspect_sample(0)

In [ ]:
len(data)
sum(d["file_label"] for d in data if d["file_label"] == 1)
set(d["repo"] for d in data)


# Task
Extract all unique file paths and count all unique commit hashes from the `data` list.

In [ ]:
unique_filepaths = set()
for item in data:
    if 'filepath' in item:
        unique_filepaths.add(item['filepath'])

unique_filepaths_list = list(unique_filepaths)

print(f"Number of unique file paths: {len(unique_filepaths_list)}")
print("First 5 unique file paths:")
for i, path in enumerate(unique_filepaths_list[:5]):
    print(f"- {path}")

# Task
Modify the data loading cell `MfqQ48vx-9Qm` to add placeholder `induce_bug` (an empty list) and `file_label` (0) to each record in the `data` list loaded from "black_repo - black_repo.csv", then re-run the modified data loading cell and the subsequent cells (`_hPnI4c0ZoDB`, `ywY6mCHFaOWm`) to resolve the `KeyError` and enable the training process to proceed.

## Add Missing Labels

### Subtask:
Modify the data loading cell (`MfqQ48vx-9Qm`) to add placeholder `induce_bug` (an empty list) and `file_label` (0) to each record in the `data` list.


**Reasoning**:
The subtask requires modifying the `data` list to include placeholder `induce_bug` and `file_label` for each record. I will update the code in cell `MfqQ48vx-9Qm` to achieve this.



In [ ]:
import pandas as pd
df = pd.read_csv('black_repo - black_repo.csv')
data = df.to_dict(orient="records")

# Add placeholder for 'induce_bug' and 'file_label' for each record
for record in data:
    record['induce_bug'] = []
    record['file_label'] = 0

**Reasoning**:
The data modification has been successfully applied to the `data` list in the previous step. To ensure the training pipeline uses this updated data structure and resolve the `KeyError`, the `DefectDataset` class and the data loaders need to be re-initialized by re-running cell `_hPnI4c0ZoDB`.



In [ ]:
#Dataset class: building samples end-to-end
#We embed lines, contextualize, build graphs, and prepare labels per file

class LineGraphSample:
    def __init__(self, x, edge_index, edge_type, line_labels, file_label, mask_lines):
        self.x = x                # [L, Feat]
        self.edge_index = edge_index  # [2, E]
        self.edge_type = edge_type    # [E]
        self.line_labels = line_labels  # [L]
        self.file_label = file_label    # scalar
        self.mask_lines = mask_lines    # boolean mask for pooling

class DefectDataset(Dataset):
    def __init__(self, records, cache=False):
        self.records = records
        self.cache = cache
        self._cache = [None]*len(records)

    def __len__(self): return len(self.records)

    def __getitem__(self, idx):
        if self.cache and self._cache[idx] is not None:
            return self._cache[idx]
        rec = self.records[idx]
        lines = rec["content"].split("\n")
        # Step 1: CodeBERT line embeddings
        with torch.no_grad():
            line_embs = embed_lines_with_codebert(lines)  # [L, 768]
        # Step 2: Contextualization
        ctx_inp = line_embs.unsqueeze(0)  # [1, L, 768]
        ctx_out = context_encoder(ctx_inp).squeeze(0)  # [L, 512]
        # Step 3: Graph construction (CPG-lite)
        edge_index, edge_type = build_cpg(lines)
        # Step 4: labels
        L = len(lines)
        line_labels = torch.zeros(L, dtype=torch.float32)
        for ln in rec["induce_bug"]:
            if 0 <= ln < L:
                line_labels[ln] = 1.0
        file_label = torch.tensor(rec["file_label"], dtype=torch.long)
        mask_lines = torch.ones(L, dtype=torch.bool)  # all lines included
        sample = LineGraphSample(ctx_out, edge_index, edge_type, line_labels, file_label, mask_lines)
        if self.cache:
            self._cache[idx] = sample
        return sample

# Split data
random.shuffle(data)
train_split = int(0.7 * len(data))
val_split = int(0.85 * len(data))
train_ds = DefectDataset(data[:train_split], cache=True)
val_ds = DefectDataset(data[train_split:val_split], cache=False)
test_ds = DefectDataset(data[val_split:], cache=False)

def collate(samples):
    # We will process each file separately (no graph batching to keep things simple in Colab)
    return samples

train_loader = DataLoader(train_ds, batch_size=1, shuffle=True, collate_fn=collate)
val_loader = DataLoader(val_ds, batch_size=1, shuffle=False, collate_fn=collate)
test_loader = DataLoader(test_ds, batch_size=1, shuffle=False, collate_fn=collate)
len(train_ds), len(val_ds), len(test_ds)

**Reasoning**:
The previous step successfully re-initialized the dataset with the updated `data` list. Now, re-running the training loop (cell `ywY6mCHFaOWm`) will allow the model to train without encountering the `KeyError` related to `induce_bug`, as per the subtask instructions.



In [ ]:
#Training loop
def train_epoch(loader):
    model.train()
    total = 0.0
    for batch in loader:
        # single-sample batches for simplicity
        sample = batch[0]
        x = sample.x.to(device)
        edge_index = sample.edge_index.to(device)
        edge_type = sample.edge_type.to(device)
        line_labels = sample.line_labels.to(device)
        file_label = sample.file_label.to(device)
        mask_lines = sample.mask_lines.to(device)

        optimizer.zero_grad()
        line_logits, file_logits = model(x, edge_index, edge_type, mask_lines)
        line_prob = torch.sigmoid(line_logits)
        line_loss = focal_loss(line_prob, line_labels)
        file_loss = F.cross_entropy(file_logits.unsqueeze(0), file_label.unsqueeze(0))
        loss = lambda_line * line_loss + lambda_file * file_loss
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        total += loss.item()
    return total / len(loader)

def evaluate(loader):
    model.eval()
    file_probs, file_labels = [], []
    line_probs, line_labels = [], []

    with torch.no_grad():
        for batch in loader:
            sample = batch[0]
            x = sample.x.to(device)
            edge_index = sample.edge_index.to(device)
            edge_type = sample.edge_type.to(device)
            mask_lines = sample.mask_lines.to(device)

            line_logits, file_logits = model(x, edge_index, edge_type, mask_lines)
            fp = torch.softmax(file_logits, dim=-1)[1].item()
            file_probs.append(fp)
            file_labels.append(sample.file_label.item())

            lp = torch.sigmoid(line_logits).detach().cpu().numpy().tolist()
            ll = sample.line_labels.detach().cpu().numpy().tolist()
            line_probs.extend(lp)
            line_labels.extend(ll)

    # File-level metrics
    file_probs_np = np.array(file_probs)
    file_labels_np = np.array(file_labels)
    file_preds_bin = (file_probs_np >= 0.5).astype(int)

    acc = accuracy_score(file_labels_np, file_preds_bin)
    f1 = f1_score(file_labels_np, file_preds_bin)
    try:
        roc = roc_auc_score(file_labels_np, file_probs_np)
    except:
        roc = float('nan')
    pr = average_precision_score(file_labels_np, file_probs_np)

    # Line-level metrics (PR-AUC)
    line_probs_np = np.array(line_probs)
    line_labels_np = np.array(line_labels)
    l_pr = average_precision_score(line_labels_np, line_probs_np)

    return {
        "file": {"acc": acc, "f1": f1, "roc": roc, "pr": pr},
        "line": {"pr": l_pr}
    }

# Train
epochs = 6
for ep in range(1, epochs+1):
    tr_loss = train_epoch(train_loader)
    val_metrics = evaluate(val_loader)
    print(f"Epoch {ep:02d} | TrainLoss {tr_loss:.4f} | "
          f"Val File(acc {val_metrics['file']['acc']:.3f}, f1 {val_metrics['file']['f1']:.3f}, "
          f"roc {val_metrics['file']['roc']:.3f}, pr {val_metrics['file']['pr']:.3f}) | "
          f"Line (pr {val_metrics['line']['pr']:.3f})")

## Modify End-to-End Test

### Subtask:
Update cell `TdEUil2QcDUC` to load code content dynamically from the first record of `train_data` instead of using a hardcoded string. This will allow the end-to-end test to use real data from `train_black.csv`.


**Reasoning**:
The subtask requires modifying cell `TdEUil2QcDUC` to dynamically load code content from `train_data` instead of using a hardcoded string. This involves accessing the `content` field of the first record in `train_data`, decoding it from bytes to a UTF-8 string, and then stripping leading/trailing whitespace. This will ensure the end-to-end test uses actual data.



In [ ]:
code = train_data[0]['content'].decode('utf-8').strip()

lines = code.split("\n")

embs = encode_lines(lines)
edges, etypes = build_cpg(lines)

print("Line embeddings:", embs.shape)
print("Edges:", edges.shape)
print("Edge types:", torch.bincount(etypes))
print("✅ FULL PIPELINE WORKING")

AttributeError: 'str' object has no attribute 'decode'

**Reasoning**:
The previous attempt to decode the 'content' field failed because it is already a string, not a byte string. The `decode('utf-8')` call should be removed. The subtask requires loading the content from `train_data` and stripping whitespace.



In [ ]:
code = train_data[0]['content'].strip()

lines = code.split("\n")

embs = encode_lines(lines)
edges, etypes = build_cpg(lines)

print("Line embeddings:", embs.shape)
print("Edges:", edges.shape)
print("Edge types:", torch.bincount(etypes))
print("✅ FULL PIPELINE WORKING")

Line embeddings: torch.Size([1, 768])
Edges: torch.Size([0])


NotImplementedError: "bincount_cpu" not implemented for 'Float'

**Reasoning**:
The 'content' field in `train_data` is a string literal of a byte string (e.g., "b'...' "), not an actual byte string, which caused incorrect splitting into lines and led to empty `etypes` being a float tensor when `torch.bincount` was called. The fix involves using `ast.literal_eval` to correctly parse the content into a bytes object before decoding, and adding a check for empty `etypes` for robustness.



In [ ]:
import ast # New import needed to safely evaluate string literals

# The 'content' field in train_data is a string representation of a byte string (e.g., "b'...' ").
# We need to use ast.literal_eval to convert this string literal into an actual bytes object,
# and then decode it to a UTF-8 string.
code_content_literal = train_data[0]['content']
code = ast.literal_eval(code_content_literal).decode('utf-8').strip()

lines = code.split("\n")

embs = encode_lines(lines)
edges, etypes = build_cpg(lines)

print("Line embeddings:", embs.shape)
print("Edges:", edges.shape)

# Check if etypes is empty before calling bincount to avoid error on empty tensor
if etypes.numel() > 0:
    # etypes should be of type long if non-empty, but explicitly cast to ensure correct type
    print("Edge types:", torch.bincount(etypes.long()))
else:
    print("Edge types: No edges generated.")
print("✅ FULL PIPELINE WORKING")

Line embeddings: torch.Size([985, 768])
Edges: torch.Size([2, 21032])
Edge types: tensor([ 1968, 12216,  6848])
✅ FULL PIPELINE WORKING
